This script contains implementation of Global (Needleman-Wunsch), Fitting and Local (Smith Waterman) alignments along with some test examples. Note: This was done as a part of course on Bioinformatics at UIUC. Parts of the code and test examples have been adapted from the course content.

In [6]:
def read_blosum62(path):
    """
    Reads in the ncbi's BLOSUM62.txt file and loads the scoring matrix
    into a dictionary.
    
    :param: path is the full path in the local filesystem at which the .txt file is located
    :return: a dictionary of dictionaries which will hold the cost of various amino acid
    substitutions as defined in BLOSUM62.
    """
    delta = {}
    with open(path, 'r') as f:
        #skipping the information in the header
        lines = f.readlines()[6:]
        keys = lines[0].split()
        
        #gap is labelled as '*' in the file, change it to standard '-'
        keys[-1] = '-'
        
        #reading the scoring information for each amino acid and storing it as a dictionary
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}  
    
    return delta


In [18]:
#Needleman-Wunsch-Global Aligment
import numpy as np
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)
point_list = [ORIGIN,UP,LEFT,TOPLEFT]

def traceback_global(v, w, pointers):
    """
    performs the traceback step of Needleman-Wunsch to obtain one possible alignment maximizing alignment score.
    There may be other possible alignments that give the same optimal edit distance.
    """
    
    i,j = len(v), len(w)
    #aligned sequences
    new_v = []
    new_w = []
    
    while True:
        di, dj = pointers[i][j]
        
        #if there is a gap in sequence v, f(i,j) = f(i,j-1) + gap penalty during forward pass
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
            
        #if there is a gap in sequence w, f(i,j) = f(i-1,j) + gap penalty during forward pass
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
            
        #if it is a match or mismatch case, f(i,j) = f(i-1,j-1) + 0 or mismatch score during forward pass
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
            
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
            
    #print(new_v, new_w)
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])
    


def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    """
    
    #edit distance for substring v(0...i) and w(0...j)
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    
    #pointers needed for traceback step to get the optimal alignment
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    
    score, alignment = None, None
    
    #intialization for boundary cases
    M[0][:] = [-j for j in range(len(w)+1)]
    pointers[0][:] = [LEFT for j in range(len(w)+1)]
    pointers[0][0] = ORIGIN
    
    #forward pass to obtian the edit distances
    for i in range(1,len(v)+1):
        for j in range(len(w)+1):
            temp = [-float("inf"),-float("inf"),-float("inf"),-float("inf")]
            if(i==0 and j==0):
                temp[0] = 0
            if(i>0):
                temp[1] = M[i-1][j] + delta[v[i-1]]['-']
            if(j>0):
                temp[2] = M[i][j-1] + delta['-'][w[j-1]]
            if(i>0 and j>0):
                temp[3] = M[i-1][j-1] + delta[v[i-1]][w[j-1]]
                    
            pointers[i][j] = point_list[np.argmax(temp)]
            M[i][j] = temp[np.argmax(temp)]
    
    #rightmost entry of bottom row gives the optimal edit distance/alignment score
    score = M[len(v)][len(w)]
    alignment = traceback_global(v,w, pointers)
    return score, alignment

In [20]:
#Test cases for protein alignment
a = 'GIVEQCCTSICSLYQLENYCN'
b = 'FVNQHLCGSHLVEALYLVCGERGFFYTPKA'
c = 'GIVEQCCASVCSLYQLENYCN'
d = 'FVNQHLCGSHLVEALYLVCGERGFFYTPKA'

blosum = read_blosum62('BLOSUM62.txt')

scoreA,alignmentA = global_align(a,c,blosum)
scoreB,alignmentB = global_align(b,d,blosum)
print(alignmentA,scoreA)
print(alignmentB, scoreB)

print(global_align('LRRGEPVY', 'LEKGDTLYILVG', blosum))
print(global_align('ALYFFT', 'QCASVT', blosum))

# Test cases for DNA alignment
# Scoring matrix (dictionary) for nucleotides
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

print(global_align('GTTTTATAGC', 'TGGATCGAT', delta))

GIVEQCCTSICSLYQLENYCN
GIVEQCCASVCSLYQLENYCN 115
FVNQHLCGSHLVEALYLVCGERGFFYTPKA
FVNQHLCGSHLVEALYLVCGERGFFYTPKA 167
(5, 'LRRGEPVY----\nLEKGDTLYILVG')
(-2, 'ALYFFT\nQCASVT')
(-3, 'GTTTTATAGC-\n-TGG-ATCGAT')


In [22]:
#Fitting alignment
import numpy as np
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)
point_list = [ORIGIN,UP,LEFT,TOPLEFT]

def traceback_fitting(v, w, M, init_j, pointers):
    """
    performs the traceback step of fitting alignment to obtain one possible alignment maximizing alignment score.
    There may be other possible alignments that give the same optimal edit distance.
    In addition to the pointers, we also need the position of the column from which to start the alignment.
    """
    i, j = len(v), init_j
    new_v = []
    new_w = []
    di,dj = (pointers[0][0])
    
    while True:
        di, dj = pointers[i][j]
        
        #if there is a gap in sequence v, f(i,j) = f(i,j-1) + gap penalty during forward pass
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
            
        #if there is a gap in sequence w, f(i,j) = f(i-1,j) + gap penalty during forward pass
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
            
        #if it is a match or mismatch case, f(i,j) = f(i-1,j-1) + 0 or mismatch score during forward pass
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def fitting_align(short, reference, delta):
    """
    Returns the score of the maximum scoring alignment of short and all 
    substrings of reference. 
    
    :param: short the shorter of the two strings we are trying to align    
    :param: reference the longer string among whose substrings we are doing global alignment
    :param: delta the scoring function for the alphabet of the two strings
    
    :returns: a tuple (score, alignment)
    """
    
    #edit distance for substring v(0...i) and w(0...j)
    M = [[0 for j in range(len(reference)+1)] for i in range(len(short)+1)]    
    
    #pointers needed for traceback step to get the optimal alignment
    pointers = [[ORIGIN for j in range(len(reference)+1)] for i in range(len(short)+1)] 
    
    score = None
    init_j = 0
    
    #forward pass to obtian the alignment score
    for i in range(1,len(short)+1):
        for j in range(len(reference)+1):
            temp = [-float("inf"),-float("inf"),-float("inf"),-float("inf")]
            if(i==0 and j==0):
                temp[0] = 0
            if(i>0):
                temp[1] = M[i-1][j] + delta[short[i-1]]['-']
            if(j>0):
                temp[2] = M[i][j-1] + delta['-'][reference[j-1]]
            if(i>0 and j>0):
                temp[3] = M[i-1][j-1] + delta[short[i-1]][reference[j-1]]
            pointers[i][j] = point_list[np.argmax(temp)]     
            M[i][j] = max(temp)
            
    #start of alignment is at the column with maximum score in the bottom column
    score = max(M[len(short)][:])
    max_idx = [i for i,j in enumerate(M[len(short)][:]) if j == score]
    init_j = max_idx[-1]
    
    alignment = traceback_fitting(short,reference,M, init_j,pointers)
    return score, alignment

In [24]:
# scoring matrix
keys = ['A', 'C', 'T', 'G', '-']
delta_fitting = {}
for i in range(len(keys)):
    delta_fitting[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_fitting['-']['-'] = -1

blosum = read_blosum62('BLOSUM62.txt')
# Test cases

print(fitting_align('ACTGCT', 'ACTGTCGTACGTGTACGTGCTATTACGATTCGGATGCATTGTGCATTTGGGCGATCTTATTCTTATC', delta))
print(fitting_align('PPPVP','MHHHHHHSSPIDPPGKPVPLNITRHTVTLKWAKPEYTGGFKITSYIVEKRDLPNGRWLKANFSNILENEFTVSGLTEDAA\
YEFRVIAKNAAGAISPPSEPSDAITCRDDVEA', blosum))

(5, 'AC-TGCT\nACGTGCT')
(24, 'PP--PVP\nPPGKPVP')


In [25]:
#Smith Waterman-Local alignment
import numpy as np
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)
point_list = [ORIGIN,UP,LEFT,TOPLEFT]

def traceback_local(v, w, M, init_i, init_j, pointers):
    i,j = init_i, init_j
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (M[i][j] == 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def local_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of all possible substrings of v and w. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]        
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)] 
    score = None
    init_i, init_j = 0,0
    for i in range(1,len(v)+1):
        for j in range(len(w)+1):
            temp = [0,-float("inf"),-float("inf"),-float("inf")]
            if(i>0):
                temp[1] = M[i-1][j] + delta[v[i-1]]['-']
            if(j>0):
                #print(i,j)
                temp[2] = M[i][j-1] + delta['-'][w[j-1]]
            if(i>0 and j>0):
                temp[3] = M[i-1][j-1] + delta[v[i-1]][w[j-1]]
            pointers[i][j] = point_list[np.argmax(temp)]       
            M[i][j] = max(temp)
    score = np.max(M)
    max_pos = np.where(M == score)
    max_idx = np.argmax(max_pos[1])
    init_i = (max_pos[0])[max_idx]
    init_j = (max_pos[1])[max_idx]
    alignment = traceback_local(v, w, M, init_i, init_j , pointers)
    return score, alignment 

In [27]:
LTK_MOUSE = "MGCSHRLLLWLGAAGTILCSNSEFQTPFLTPSLLPVLVLNSQEQKVTPTP\
SKLEPASLPNPLGTRGPWVFNTCGASGRSGPTQTQCDGAYTGSSVMVTVG\
AAGPLKGVQLWRVPDTGQYLISAYGAAGGKGAQNHLSRAHGIFLSAVFFL\
RRGEPVYILVGQQGQDACPGGSPESQLVCLGESGEHATTYGTERIPGWRR\
WAGGGGGGGGATSIFRLRAGEPEPLLVAAGGGGRSYRRRPDRGRTQAVPE\
RLETRAAAPGSGGRGGAAGGGSGWTSRAHSPQAGRSPREGAEGGEGCAEA\
WAALRWAAAGGFGGGGGACAAGGGGGGYRGGDTSESDLLWADGEDGTSFV\
HPSGELYLQPLAVTEGHGEVEIRKHPNCSHCPFKDCQWQAELWTAECTCP\
EGTELAVDNVTCMDLPTTASPLILMGAVVAALALSLLMMCAVLILVNQKC\
QGLWGTRLPGPELELSKLRSSAIRTAPNPYYCQVGLSPAQPWPLPPGLTE\
VSPANVTLLRALGHGAFGEVYEGLVTGLPGDSSPLPVAIKTLPELCSHQD\
ELDFLMEALIISKFSHQNIVRCVGLSFRSAPRLILLELMSGGDMKSFLRH\
SRPHPGQLAPLTMQDLLQLAQDIAQGCHYLEENHFIHRDIAARNCLLSCS\
GASRVAKIGDFGMARDIYQASYYRKGGRTLLPVKWMPPEALLEGLFTSKT\
DSWSFGVLLWEIFSLGYMPYPGHTNQEVLDFIATGNRMDPPRNCPGPVYR\
IMTQCWQHQPELRPDFGSILERIQYCTQDPDVLNSPLPVEPGPILEEEEA\
SRLGNRSLEGLRSPKPLELSSQNLKSWGGGLLGSWLPSGLKTLKPRCLQP\
QNIWNPTYGSWTPRGPQGEDTGIEHCNGSSSSSIPGIQ"

ATK_MOUSE = "MGAAGFLWLLPPLLLAAASYSGAATDQRAGSPASGPPLQPREPLSYSRLQ\
RKSLAVDFVVPSLFRVYARDLLLPQPRSPSEPEAGGLEARGSLALDCEPL\
LRLLGPLPGISWADGASSPSPEAGPTLSRVLKGGSVRKLRRAKQLVLELG\
EETILEGCIGPPEEVAAVGILQFNLSELFSWWILHGEGRLRIRLMPEKKA\
SEVGREGRLSSAIRASQPRLLFQIFGTGHSSMESPSETPSPPGTFMWNLT\
WTMKDSFPFLSHRSRYGLECSFDFPCELEYSPPLHNHGNQSWSWRHVPSE\
EASRMNLLDGPEAEHSQEMPRGSFLLLNTSADSKHTILSPWMRSSSDHCT\
LAVSVHRHLQPSGRYVAQLLPHNEAGREILLVPTPGKHGWTVLQGRVGRP\
ANPFRVALEYISSGNRSLSAVDFFALKNCSEGTSPGSKMALQSSFTCWNG\
TVLQLGQACDFHQDCAQGEDEGQLCSKLPAGFYCNFENGFCGWTQSPLSP\
HMPRWQVRTLRDAHSQGHQGRALLLSTTDILASEGATVTSATFPAPMKNS\
PCELRMSWLIRGVLRGNVSLVLVENKTGKEQSRTVWHVATDEGLSLWQHT\
VLSLLDVTDRFWLQIVTWWGPGSRATVGFDNISISLDCYLTISGEEKMSL\
NSVPKSRNLFEKNPNKESKSWANISGPTPIFDPTVHWLFTTCGASGPHGP\
TQAQCNNAYQNSNLSVVVGSEGPLKGVQIWKVPATDTYSISGYGAAGGKG\
GKNTMMRSHGVSVLGIFNLEKGDTLYILVGQQGEDACPRANQLIQKVCVG\
ENNVIEEEIRVNRSVHEWAGGGGGGGGATYVFKMKDGVPVPLIIAAGGGG\
RAYGAKTETFHPERLESNSSVLGLNGNSGAAGGGGGWNDNTSLLWAGKSL\
LEGAAGGHSCPQAMKKWGWETRGGFGGGGGGCSSGGGGGGYIGGNAASNN\
DPEMDGEDGVSFISPLGILYTPALKVMEGHGEVNIKHYLNCSHCEVDECH\
MDPESHKVICFCDHGTVLADDGVSCIVSPTPEPHLPLSLILSVVTSALVA\
ALVLAFSGIMIVYRRKHQELQAMQMELQSPEYKLSKLRTSTIMTDYNPNY\
CFAGKTSSISDLKEVPRKNITLIRGLGHGAFGEVYEGQVSGMPNDPSPLQ\
VAVKTLPEVCSEQDELDFLMEALIISKFNHQNIVRCIGVSLQALPRFILL\
ELMAGGDLKSFLRETRPRPNQPTSLAMLDLLHVARDIACGCQYLEENHFI\
HRDIAARNCLLTCPGAGRIAKIGDFGMARDIYRASYYRKGGCAMLPVKWM\
PPEAFMEGIFTSKTDTWSFGVLLWEIFSLGYMPYPSKSNQEVLEFVTSGG\
RMDPPKNCPGPVYRIMTQCWQHQPEDRPNFAIILERIEYCTQDPDVINTA\
LPIEYGPVVEEEEKVPMRPKDPEGMPPLLVSPQPAKHEEASAAPQPAALT\
APGPSVKKPPGAGAGAGAGAGAGPVPRGAADRGHVNMAFSQPNPPPELHK\
GPGSRNKPTSLWNPTYGSWFTEKPAKKTHPPPGAEPQARAGAAEGGWTGP\
GAGPRRAEAALLLEPSALSATMKEVPLFRLRHFPCGNVNYGYQQQGLPLE\
ATAAPGDTMLKSKNKVTQPGP"

score_local, alignment_local = local_align(LTK_MOUSE, ATK_MOUSE, blosum)
print(score_local)
#print(alignment_local)

2355


In [29]:
keys = ['A', 'C', 'T', 'G', '-']
delta_local = {}
for i in range(len(keys)):
    delta_local[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_local['-']['-'] = -1
print(local_align("TACGTGACGTCTATCAT", "TTGTGCATGTATCTGAC", delta_local))
print(local_align("ACTGATTTCGATGCTGTACGTGACGTACGTTTATTCTATCAT", "TCTGACTGTGACTATACTATTGTGCATGTATCTGACTAGCTAG", delta_local))


(6, 'GTGACGTCTATCAT\nGTG-CATGTATC-T')
(13, 'ACTGATTTCGATGC-TGTA-CGTGACGTACGTTTA\nACT-ATT--G-TGCATGTATC-TGAC-TA-GCT-A')
